In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import imgkit


def get_ranked_rows(df, n_argmax, col_keyname):
    print(col_keyname)
    top_n = {}
    for i in df[col_keyname]:
        idx = df.loc[df[col_keyname] == i, df.columns != col_keyname].abs().values.argsort(1)[:, -n_argmax:][:, ::-1]
        idx += 1
        cols = df.columns[idx][0]
        for col in cols:
            if col in top_n.keys():
                top_n[col] += 1
            else:
                top_n[col] = 1
    return top_n

def highlight_max(s):    
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):    
    is_max = s == s.min()
    return ['color: white' if v else '' for v in is_max]


In [46]:
# Initialize and clean the results
df = pd.read_csv('/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/results_ucr.csv')
cols=[i for i in df.columns if i not in ["dataset"]]
for col in cols:
    df[col]=pd.to_numeric(df[col])

# Save best algo on each run
top1 = df.loc[:, df.columns != 'dataset'].idxmax(axis=1)
top1 = dict(top1.value_counts())
cols = [k for k,v in top1.items() if v >= 5]
cols.append('dataset')
df_check = df[cols]

# Add state of the art from (https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline)
df_soa = pd.read_csv('/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/stateofart.csv')
df_soa = df_soa.rename(columns={'Unnamed: 0':'dataset'}) 
df_soa = df_soa.replace('/', np.nan)
cols=[i for i in df_soa.columns if i not in ["dataset"]]

# Have "error rate" which I assume is equivalent to: 1 - acc
for col in cols:
    df_soa[col] = 1 - pd.to_numeric(df_soa[col],errors='coerce')
df_soa = df_soa.replace(np.nan, 0)

# Add info on the dataset
df_summary = pd.read_csv('/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/DataSummary.csv')
df_summary = df_summary.rename(columns={'Name':'dataset'})
df_summary = df_summary[['Type', 'dataset', 'Train ', 'Test ', 'Class']]
df_stateofArt = df_check.join(df_soa.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.join(df_summary.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.drop(['Train ', 'Test ', 'Class'], axis =1)


In [44]:
df_type = df_stateofArt.groupby('Type').mean()
table = df_type.style.background_gradient(cmap='viridis_r', axis = 1).apply(highlight_max, axis = 1)
table
html = table.render()
imgkit.from_string(html, '/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/SOTA_categories.png')

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [45]:
df_stateofArt = df_check.join(df_soa.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.join(df_summary.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.drop(['Train ', 'Test ','Type'], axis =1)
df_type = df_stateofArt.groupby('Class').mean()
table = df_type.style.background_gradient(cmap='viridis_r', low = .1).apply(highlight_max, axis = 1).apply(highlight_min, axis=1)

In [42]:
df_stateofArt = df_check.join(df_soa.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.join(df_summary.set_index('dataset'), on='dataset')
df_stateofArt['groups']=pd.cut(df_stateofArt['Train '],[0,50,100,200,300,500,999])
df_stateofArt = df_stateofArt.drop(['Class', 'Test ','Train ','Type'], axis =1)
df_type = df_stateofArt.groupby('groups').mean()
table = df_type.style.background_gradient(cmap='viridis_r', axis = 1).apply(highlight_max, axis = 1)
table
html = table.render()
imgkit.from_string(html, '/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/SOTA_lengths.png')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [160]:
# Join the dataframes and see what performs best... note some names don't match
df_stateofArt = df_check.join(df_soa.set_index('dataset'), on='dataset')
df_stateofArt = df_stateofArt.set_index('dataset', drop = True)
table = df_stateofArt.iloc[:,1::].style.apply(highlight_max, axis = 1)
html = table.render()
imgkit.from_string(html, '/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/SOTA.png')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [161]:
cm = sns.light_palette("green", as_cmap=True)
table = df_stateofArt.style.background_gradient(cmap='viridis_r', low = .1, axis = 1).apply(highlight_max, axis = 1).apply(highlight_min, axis=1)
html = table.render()
imgkit.from_string(html, '/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/SOTA_colorROW.png')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [162]:
cm = sns.light_palette("green", as_cmap=True)
table = df_stateofArt.style.background_gradient(cmap='viridis_r', low = .1).apply(highlight_max, axis = 1).apply(highlight_min, axis=1)
html = table.render()
imgkit.from_string(html, '/Users/ezekielbarnett/Documents/canfield/diffusion/results/ucr/SOTA_color.png')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True